In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as Fa,b
import torch.optim as optim
from torchvision import datasets, transforms

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print('Current cuda device is', device)

Current cuda device is cpu


In [ ]:
train_data = datasets.MNIST(root = './data', train =True, download = True, transform = transforms.ToTensor())
test_data = datasets.MNIST(root = './data', train =False, download = True, transform = transforms.ToTensor())

print(len(train_data))
print(len(test_data))

60000
10000


In [ ]:
fig = plt.figure(figsize=(8,8))

row, col = (3,3)

for i in range(1, row*col +1):
  idx = torch.randint(len(train_data), size=(1,)).item()
  img, label = train_data[idx]
  fig.add_subplot(row, col, i)
  plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
ds = datasets.FashionMNIST(root='./data', train=True, download = True, 
                           transform = transforms.ToTensor(), 
                           target_transform=transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1)))

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
x = torch.rand(1,28,28, device = device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

tensor([6])


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
seq_models = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_models(input_image)

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(logits.size(),"\n", logits)
print(pred_probab.size(),"\n", pred_probab)

torch.Size([3, 10]) 
 tensor([[ 0.0875, -0.0952,  0.0321,  0.1969, -0.0486,  0.2574,  0.0686,  0.2287,
         -0.0918, -0.0711],
        [ 0.0751, -0.0675,  0.0075,  0.2269,  0.0220,  0.2230, -0.0266,  0.1215,
         -0.1011, -0.1147],
        [-0.0189,  0.0313,  0.0409,  0.2154,  0.0406,  0.1907,  0.1668,  0.1633,
         -0.1568, -0.0676]], grad_fn=<AddmmBackward0>)
torch.Size([3, 10]) 
 tensor([[0.1023, 0.0852, 0.0968, 0.1141, 0.0893, 0.1213, 0.1004, 0.1178, 0.0855,
         0.0873],
        [0.1032, 0.0895, 0.0965, 0.1201, 0.0979, 0.1196, 0.0932, 0.1081, 0.0865,
         0.0854],
        [0.0917, 0.0965, 0.0974, 0.1160, 0.0974, 0.1131, 0.1105, 0.1101, 0.0799,
         0.0874]], grad_fn=<SoftmaxBackward0>)


In [ ]:
print(model)
for name, param in model.named_parameters():
  print(f"\n{name},\n {param.size()},\n {param[:2]}")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

linear_relu_stack.0.weight,
 torch.Size([512, 784]),
 tensor([[ 0.0086,  0.0033, -0.0326,  ...,  0.0318,  0.0052, -0.0097],
        [-0.0027, -0.0214,  0.0331,  ...,  0.0237, -0.0190,  0.0070]],
       grad_fn=<SliceBackward0>)

linear_relu_stack.0.bias,
 torch.Size([512]),
 tensor([-0.0293, -0.0215], grad_fn=<SliceBackward0>)

linear_relu_stack.2.weight,
 torch.Size([512, 512]),
 tensor([[-0.0298,  0.0215,  0.0396,  ..., -0.0388,  0.0025,  0.0011],
        [ 0.0394,  0.0265, -0.0268,  ..., -0.0108, -0.0143,  0.0162]],
       grad_fn=<SliceBackward0>)

linear_relu_stack.2.bias,
 torch.Size([512]),
 tensor([ 0.0110, -0.0342], grad_fn=<SliceBackward0>)

linear_relu_stack.4.